In [1]:
import requests
import csv
import os
import pandas as pd
import json
import re
import time
from datetime import datetime, timedelta
import math
import traceback

In [17]:
input_path = "C:\\Users\\khan32\\Documents\\pr_fg_exchange\\"
output_path = input_path

In [19]:
url = 'https://database.factgrid.de/sparql'
query = (
"""SELECT ?item ?prid WHERE {
  ?item wdt:P472 ?prid.
}""")
# SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }

r = requests.get(url, params={'query': query}, headers={"Accept": "application/json"})
data = r.json()
factgrid_df = pd.json_normalize(data['results']['bindings'])

len(factgrid_df)

5405

In [20]:
factgrid_df

,item.type,item.value,prid.type,prid.value
0,uri,https://database.factgrid.de/entity/Q652063,literal,059-00013-001
1,uri,https://database.factgrid.de/entity/Q652072,literal,005-00027-001
2,uri,https://database.factgrid.de/entity/Q652112,literal,006-00006-001
3,uri,https://database.factgrid.de/entity/Q652104,literal,014-00007-001
4,uri,https://database.factgrid.de/entity/Q652076,literal,035-00015-001
...,...,...,...,...
5400,uri,https://database.factgrid.de/entity/Q730735,literal,060-00325-001
5401,uri,https://database.factgrid.de/entity/Q730738,literal,063-00704-001
5402,uri,https://database.factgrid.de/entity/Q730777,literal,305-00265-001
5403,uri,https://database.factgrid.de/entity/Q730807,literal,305-00048-001


In [21]:
# extract out q id
def extract_qid(df, column):
    df[column] = df[column].map(lambda x: x.strip('https://database.factgrid.de/entity/'))
 
#factgrid_df['item.value'] = factgrid_df['item.value'].map(lambda x: x.strip('https://database.factgrid.de/entity/'))

# drop irrelevant columns
def drop_type_columns(df):
    df.drop(columns=[column for column in df.columns if column.endswith('type')], inplace=True)
    df.drop(columns=[column for column in df.columns if column.endswith('xml:lang')], inplace=True)

In [22]:
drop_type_columns(factgrid_df)
extract_qid(factgrid_df, 'item.value')
factgrid_df.columns = ['FactGrid_ID', 'pd_id']
factgrid_df

,FactGrid_ID,pd_id
0,Q652063,059-00013-001
1,Q652072,005-00027-001
2,Q652112,006-00006-001
3,Q652104,014-00007-001
4,Q652076,035-00015-001
...,...,...
5400,Q730735,060-00325-001
5401,Q730738,063-00704-001
5402,Q730777,305-00265-001
5403,Q730807,305-00048-001



run this query on personendatenbank and export the results to a csv with the default export settings on phpmyadmin

```sql
SELECT persons.factgrid, persons.id, gsn.nummer 
FROM items INNER JOIN persons ON persons.item_id = items.id AND persons.deleted=0 AND items.deleted=0 AND items.status = "online" 
INNER JOIN gsn ON gsn.item_id = items.id AND gsn.deleted=0 
```

In [50]:
filename = 'persons_2024-07-03.csv'

In [52]:
pr_df = pd.read_csv(os.path.join(input_path, filename), names=["fg_id", "id", "pd_id"])
pr_df

,fg_id,id,pd_id
0,NaN,281468,002-03990-001
1,NaN,281467,002-03989-001
2,NaN,281463,002-03985-001
3,NaN,281460,002-03982-001
4,NaN,281453,002-03975-001
...,...,...,...
101689,Q653267,336406,007-03087-001
101690,NaN,336408,027-04097-001
101691,NaN,336408,033-02524-001
101692,Q654054,336410,060-02172-001


In [53]:
joined_df = factgrid_df.merge(pr_df, how='outer', on='pd_id', suffixes=('_wiag', '_pd'), indicator=True)
joined_df

,FactGrid_ID,pd_id,fg_id,id,_merge
0,NaN,002-00002-001,NaN,277487,right_only
1,NaN,002-00003-001,NaN,277488,right_only
2,NaN,002-00007-001,NaN,277493,right_only
3,NaN,002-00007-002,NaN,309774,right_only
4,NaN,002-00009-001,Q652072,320471,right_only
...,...,...,...,...,...
101693,NaN,713-00847-001,Q653704,332975,right_only
101694,NaN,713-00848-001,NaN,335132,right_only
101695,NaN,713-00850-001,NaN,335119,right_only
101696,NaN,713-00851-001,NaN,335120,right_only


The output of the cell below should be empty. The cell after it will produce links to the entries on personendatenbank if this is not the case

In [54]:
joined_df[joined_df['_merge'] == 'left_only']

,FactGrid_ID,pd_id,fg_id,id,_merge


In [55]:
for _, row in joined_df[joined_df['_merge'] == 'left_only'].iterrows():
    print('https://personendatenbank.germania-sacra.de/index/gsn/' + row['pd_id'])

In [58]:
join_df = joined_df[joined_df['_merge'] == 'both']
join_df

,FactGrid_ID,pd_id,fg_id,id,_merge
26,Q652124,002-00033-001,Q652124,277541,both
72,Q652142,002-00041-001,Q652142,277547,both
74,Q652453,002-00041-003,Q652453,291427,both
148,Q653809,002-00118-003,Q653809,291433,both
169,Q652246,002-00155-001,Q652246,291424,both
...,...,...,...,...,...
101443,Q652738,713-00366-001,Q652738,334499,both
101563,Q653198,713-00604-001,Q653198,334852,both
101586,Q653230,713-00636-001,Q653230,334899,both
101596,Q653286,713-00658-001,Q653286,334920,both


In [60]:
unequal_df = join_df[join_df['FactGrid_ID'] != join_df['fg_id']]
unequal_df

,FactGrid_ID,pd_id,fg_id,id,_merge
1955,Q883810,002-02448-001,NaN,320592,both
2294,Q883864,002-02973-002,NaN,330901,both
4297,Q947409,003-01745-001,NaN,6281,both
4805,Q948993,005-00125-001,NaN,331033,both
4948,Q947607,005-00799-001,NaN,7908,both
...,...,...,...,...,...
99820,Q948490,700-00306-001,NaN,330773,both
99919,Q883809,700-00405-001,NaN,319177,both
100243,Q948520,700-00807-001,NaN,330856,both
101115,Q883874,712-00335-001,NaN,332058,both


generate links to check for possible duplicates on factgrid

In [66]:
linkify = lambda x : 'https://database.factgrid.de/wiki/Item:' + x 
for _, row in unequal_df.iterrows():
    if not pd.isna(row['fg_id']):
        print(linkify (row['FactGrid_ID']), linkify (row['fg_id']))

https://database.factgrid.de/wiki/Item:Q883667 https://database.factgrid.de/wiki/Item:Q883772
https://database.factgrid.de/wiki/Item:Q883853 https://database.factgrid.de/wiki/Item:Q402050
https://database.factgrid.de/wiki/Item:Q652674 https://database.factgrid.de/wiki/Item:Q652673
https://database.factgrid.de/wiki/Item:Q900423 https://database.factgrid.de/wiki/Item:Q401992
https://database.factgrid.de/wiki/Item:Q721999 https://database.factgrid.de/wiki/Item:Q401703


In [69]:
today_string = datetime.now().strftime('%Y-%m-%d')

In [70]:
query = "LOCK TABLES persons WRITE;\n"
for _, row in unequal_df.iterrows():
    query += f"""
    UPDATE persons
    SET factgrid = '{row['FactGrid_ID']}'
    WHERE id = {row['id']}; -- id: {row['pd_id']}
"""
query += "\nUNLOCK TABLES;"
with open(os.path.join(output_path, f'update_pr_fg_ids_{today_string}.sql'), 'w') as file:
    file.write(query)